In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools

In [2]:
def create_cpt(var_names, probs, levels_list):
    cpt = pd.DataFrame(columns=["probs"] + var_names)
    
    cpt["probs"] = probs
    
    levels_table = list(itertools.product(*(levels_list)))
    
    if not len(levels_table) == len(probs):
        raise('ERROR:: Incorrect probability dimensions')
        return None
    
    cpt[var_names] = levels_table
    return cpt

In [3]:
def observe(network, var_name, observation):
    observed_network = []
    for cpt in network:
        for var,obs in zip(var_name, observation):
            if var in cpt.columns:
                cpt = cpt[cpt[var] == obs]
        observed_network.append(cpt)
        
    return observed_network

In [4]:
def marginalize_factor(cpt, factor):
    probs = cpt["probs"]
    cpt = cpt.drop([factor, "probs"], axis=1)
    
    marginalized_cpt = pd.DataFrame(columns=list(cpt.columns))
    marginalized_probs = []
    while cpt.shape[0] > 0:
        positions = [x for x in range(0, cpt.shape[0]) if sum(cpt.iloc[0] == cpt.iloc[x]) == cpt.shape[1]]
        marginalized_probs.append(sum(probs[probs.index[positions]]))
        
        marginalized_cpt = marginalized_cpt.append(cpt[:1])
        
        cpt = cpt.drop(cpt.index[positions], axis=0)
        probs = probs.drop(probs.index[positions], axis=0)
    
    marginalized_cpt.insert(0, "probs", marginalized_probs)
    return marginalized_cpt

In [5]:
def marginalize_factors(cpt, factors):
    for factor in factors:
        cpt = marginalize_factor(cpt, factor)
    
    return cpt

In [6]:
def marginalize(network, factors):
    marginalized_network = []
    for cpt in network:
        cpt = marginalize_factors(cpt, factors)
        marginalized_network.append(cpt)
    return marginalized_network

In [7]:
def factor_product(cpt1, cpt2):
    column_names = list(set(cpt1.columns).union(set(cpt2.columns)).difference(set(["probs"])))
    shared_features = list(set(cpt1.columns).intersection(set(cpt2.columns)).difference(set(["probs"])))
    probs = []
    
    cpt = pd.DataFrame(columns=column_names)
    idx = 0
    
    for i in range(0, cpt1.shape[0]):
        for j in range(0, cpt2.shape[0]):
            if sum(cpt1[shared_features].iloc[i] == cpt2[shared_features].iloc[j]) == len(shared_features):
                probs.append(cpt1["probs"].iloc[i] * cpt2["probs"].iloc[j])
                vals = []
                for name in column_names:
                    if name in cpt1.columns:
                        vals.append(cpt1[name].iloc[i])
                    else:
                        vals.append(cpt2[name].iloc[j])
                cpt.loc[idx] = vals
                idx = idx + 1
                
    cpt.insert(0, "probs", probs)
    return cpt

In [8]:
def infer(network, marg_vars, obs_vars, obs_vals):
    network = observe(network, obs_vars, obs_vals)
    
    for marg_var in marg_vars:
        marg_net = []
        rm_nodes = []
        for i, node in enumerate(network):
            if marg_var in node.columns:
                marg_net.append(node)
                rm_nodes.append(i)
                
        if not len(marg_net) == 0:
            table = marg_net[0]
            
            rm_nodes.reverse()
            for rm in rm_nodes:
                del network[rm]
                
            for cpt in marg_net[1:]:
                table = factor_product(table, cpt)
                
            marginalized_table = marginalize_factor(table, marg_var)
            network.append(marginalized_table)            
    
    # When all variables have been marginalized product the table together
    product = network[0]
    for node in network[1:]:
        product = factor_product(product, node)
        
    # Normalize
    product["probs"] = product["probs"] / sum(product["probs"])
    
    return product

![Model](http://www.nbertagnolli.com/assets/Bayes_Nets/figure_01.png)

In [10]:
chol = create_cpt(["s", "c"], [.8, .2, .4, .6], [(1, 0), (1, 0)])
smoke = create_cpt(["s"], [.15, .85], [(1, 0)])
bp = create_cpt(["e", "s", "b"], 
                [.45, .55, .05, .95, .95, .05, .55, .45], 
                [(1, 0), (1, 0), (1, 0)])
exercise = create_cpt(["e"], [.4, .6], [(1, 0)])
attack = create_cpt(["b", "a"], [.75, .25, .05, .95], [(1, 0), (1, 0)])
heart_net = [exercise, smoke, chol, bp, attack]

In [11]:
exercise

,probs,e
0,0.4,1
1,0.6,0


In [12]:
"""
Xác suất bệnh nhân bị đau tim khi biết anh ta hút thuốc
"""
infer(heart_net, ["c", "b", "e", "s"], ["s"], [1])

,probs,a
0,0.575,1
1,0.425,0


In [13]:
"""
Xác suất bệnh nhân huyết áp cao khi biết người đó bị đau tim
"""
infer(heart_net, ["c", "a", "e", "s"], ["a"], [1])

,probs,b
0,0.912463,1
1,0.087537,0


In [14]:
"""
Xác suât bệnh nhân hút thuốc và tập thể dục khi biết người đó bị huyết áp cao
"""
infer(heart_net, ["c", "a", "b"], ["b"], [1])

,probs,s,e
0,0.065854,1,1
1,0.041463,0,1
2,0.208537,1,0
3,0.684146,0,0


In [16]:
"""
Xác suất người đó hút thuốc khi biết người đó bị huyết áp cao và tập thể dục
"""
infer(heart_net, ["c", "a", "b", "e"], ["b", "e"], [1, 1])

,probs,s
0,0.613636,1
1,0.386364,0
